In [24]:
from carla_utils import get_homogeneous_matrix

In [25]:
import os
import pandas as pd
import numpy as np

# Define the path to the data directory
data_dir = "/home/sid/scans/DATA_2023-04-16_23-36-03"

# Load the GPS data into a pandas dataframe
gps_files = os.listdir(os.path.join(data_dir, "gps"))
gps_data = []
for file in gps_files:
    if file.endswith(".npy"):
        file_path = os.path.join(data_dir, "gps", file)
        gps = np.load(file_path, allow_pickle=True).item()
        location = gps['location']
        rotation = gps['rotation']
        gps_data.append([file[:-4], location['x'], location['y'], location['z'], rotation['pitch'], rotation['yaw'], rotation['roll']])
gps_df = pd.DataFrame(gps_data, columns=['file', 'x', 'y', 'z', 'pitch', 'yaw', 'roll'])

# Load the LiDAR data into a list
lidar_files = os.listdir(os.path.join(data_dir, "lidar"))
lidar_data = [file[:-4] for file in lidar_files if file.endswith(".ply")]

# Create a pandas dataframe with the LiDAR file names
lidar_df = pd.DataFrame(lidar_data, columns=['file'])

# Merge the GPS and LiDAR dataframes on the 'file' column
merged_df = pd.merge(gps_df, lidar_df, on='file')

# Display the merged dataframe
print(merged_df.head())


   file           x          y         z     pitch         yaw          roll
0  1082  103.140175  47.581596 -0.007478 -0.000034 -103.306015  4.866604e-07
1   272 -114.265587  52.219284 -0.007260  0.005437   90.039650  9.117827e-06
2   912  106.394775  64.903008 -0.006505 -0.087399  -88.183609  6.542762e-01
3   610   73.639214  70.202560 -0.007429  0.011816   -0.201660  2.774984e-02
4   752  -10.704570  15.123018 -0.007495  0.009221 -176.786530 -1.791382e-02


In [26]:
merged_df['file'] = merged_df['file'].astype(int)
merged_df = merged_df.sort_values(by='file')
merged_df = merged_df.reset_index(drop=True)
merged_df

,file,x,y,z,pitch,yaw,roll
0,191,-114.232773,43.821014,0.585300,0.000000,90.642235,0.000000e+00
1,192,-114.232773,43.821014,0.546100,0.000000,90.642235,0.000000e+00
2,193,-114.232773,43.821014,0.482400,0.000000,90.642235,0.000000e+00
3,194,-114.232773,43.821014,0.394200,0.000000,90.642235,0.000000e+00
4,195,-114.232773,43.821014,0.281500,0.000000,90.642235,0.000000e+00
...,...,...,...,...,...,...,...
1510,1701,103.140175,47.581596,-0.007478,-0.000034,-103.306015,4.866604e-07
1511,1702,103.140175,47.581596,-0.007478,-0.000034,-103.306015,4.866604e-07
1512,1703,103.140175,47.581596,-0.007478,-0.000034,-103.306015,4.866604e-07
1513,1704,103.140175,47.581596,-0.007478,-0.000034,-103.306015,4.866604e-07


In [27]:
# Create a list of dictionaries for consecutive file pairs
pairs_list = []
for i in range(1, len(merged_df)):
    # Extract the file names from the current and previous rows
    source_file = merged_df.iloc[i]['file']
    target_file = merged_df.iloc[i-1]['file']
    # Add the file names to the pairs list as a dictionary
    pairs_list.append({'source': source_file, 'target': target_file})

# Create the consecutive pairs dataframe from the pairs list
consecutive_pairs = pd.DataFrame(pairs_list)

# Display the consecutive pairs dataframe
print(consecutive_pairs.head())

   source  target
0   192.0   191.0
1   193.0   192.0
2   194.0   193.0
3   195.0   194.0
4   196.0   195.0


In [32]:
homogeneous_matrices = []
# Loop through the rows in the consecutive_pairs dataframe
for i, row in consecutive_pairs.iterrows():
    # Extract the pose information for the source and target files
    source_pose = merged_df.loc[merged_df['file'] == row['source']].values[0][1:]
    target_pose = merged_df.loc[merged_df['file'] == row['target']].values[0][1:]
    # Call the get_homogeneous_matrix() function with the extracted pose information
    homogeneous_matrix = get_homogeneous_matrix(source_pose, target_pose)
    homogeneous_matrices.append(homogeneous_matrix)

consecutive_pairs['homogeneous_matrix'] = homogeneous_matrices
consecutive_pairs.head()

,source,target,homogeneous_matrix
0,192.0,191.0,"[[1.0, 0.0, 1.2470807998616975e-17, -4.8885502..."
1,193.0,192.0,"[[1.0, 0.0, 1.2470807998616975e-17, -7.9439183..."
2,194.0,193.0,"[[1.0, 0.0, 1.2470807998616975e-17, -1.0999253..."
3,195.0,194.0,"[[1.0, 0.0, 1.2470807998616975e-17, -1.4054587..."
4,196.0,195.0,"[[1.0, 0.0, 1.2470807998616975e-17, -1.7109948..."
